In [1]:
import pandas as pd
import numpy as np
from usefuls import atx_zip_codes, census_key, zipcode_tabulation_area
import requests
from census import Census

# Prepare dictionary of census data keyed by year
# We use what years are available: 2011 - 2016
# 2010 gives geography error; zipcode geography was not supported in that year
years = list(range(2011,2017))
# Census() results
cen = dict.fromkeys(years)
# cen.acs5.get() results
census_data = dict.fromkeys(years)
# Dataframe


In [2]:
# Get census data
for y in years:
    cen[y] = Census(census_key, year=y)
    census_data[y] = cen[y].acs5.get(("B01003_001E"), {'for': zipcode_tabulation_area})

# Add year to each row-to-by
for y in years:
    for i, l in enumerate(census_data[y]):
            census_data[y][i]['year'] = y

# Convert census data to one dataframe
census_df = pd.DataFrame()
for y in years:
    census_df = census_df.append(pd.DataFrame(census_data[y]))

census_df = census_df.rename(columns={"B01003_001E": "Population", "zip code tabulation area": "Zip Code"})

In [3]:
# Convert population and Zip Code to integers
census_df['Zip Code'] = census_df['Zip Code'].apply(lambda x : int(x))

In [4]:
census_df['Population'] = census_df['Population'].apply(lambda x : int(x))

In [1]:
census_df.head()

NameError: name 'census_df' is not defined

In [6]:
reports_df = pd.read_csv('atx_crime_types_2009-2018.csv')

In [7]:
# Use only years 2011 - 2016
reports_df = reports_df[(reports_df['year'] >= 2011) & (reports_df['year'] <= 2016)]

In [8]:
# What we are working with so far
reports_df.head()

,year,Zip Code,Highest Offense Description,Reports
729,2011,78610,Burglary,1
730,2011,78610,Other,5
731,2011,78610,Theft,1
732,2011,78613,Aggravated Assault,23
733,2011,78613,Burglary,16


In [9]:
reports_df = reports_df.drop(labels='Highest Offense Description', axis=1)

In [10]:
reports_df = reports_df.groupby(['year','Zip Code']).sum().reset_index()

In [11]:
crimerate_df = reports_df.merge(census_df, on=['year', 'Zip Code'])

In [12]:
# Calculate Crime Rate per 100,000 people
crimerate_df['Crime Rate per 100,000 people'] = round(crimerate_df['Reports']/crimerate_df['Population'] * 100_000, 1)

In [13]:
crimerate_df.head()

,year,Zip Code,Reports,Population,"Crime Rate per 100,000 people"
0,2011,78610,7,22052,31.7
1,2011,78613,449,63901,702.6
2,2011,78617,1094,18572,5890.6
3,2011,78652,33,4256,775.4
4,2011,78653,97,14631,663.0


In [15]:
crimerate_df.to_csv('atx_crime_rates_2011-2016.csv', index=False)